<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Options_Trading_Strategy_based_on_ARIMA_Forecasting_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import yfinance as yf
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import *
from ibapi.wrapper import EWrapper
from statsmodels.tsa.arima.model import ARIMA

class TradingApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.nextOrderId = 0
        self.data = {}  # Store option data

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextOrderId = orderId

    def contractDetails(self, reqId: int, contractDetails):
        super().contractDetails(reqId, contractDetails)
        contract = contractDetails.contract
        symbol = contract.symbol
        right = contract.right
        self.data[symbol] = {"right": right, "strike": contract.strike}

    def tickPrice(self, reqId, tickType, price, attrib):
        super().tickPrice(reqId, tickType, price, attrib)
        if tickType == 1:  # Last price
            symbol = self.data[reqId]["symbol"]
            right = self.data[reqId]["right"]
            strike = self.data[reqId]["strike"]
            print(f"Received last price for {symbol}: {price}")

            # Implement your ARIMA forecasting logic here
            data = self.prepare_data(symbol)
            model = self.fit_arima_model(data)
            forecasted_values = self.generate_forecast(model)

            # Implement your buy and sell order logic based on the forecasted values
            if price > forecasted_values[-1] + threshold:
                self.placeOrder(symbol, right, strike, "SELL", quantity)

            if price < forecasted_values[-1] - threshold:
                self.placeOrder(symbol, right, strike, "BUY", quantity)

    def prepare_data(self, symbol):
        # Fetch historical data from Yahoo Finance
        ticker = yf.Ticker(symbol)
        data = ticker.history(period="1y")

        # Clean and preprocess the data as needed
        cleaned_data = self.clean_data(data)
        preprocessed_data = self.preprocess_data(cleaned_data)

        # Return the prepared data for modeling
        return preprocessed_data

    def clean_data(self, data):
        # Implement your data cleaning logic here
        cleaned_data = data.copy()
        # Perform data cleaning operations

        return cleaned_data

    def preprocess_data(self, data):
        # Implement your data preprocessing logic here
        preprocessed_data = data.copy()
        # Perform data preprocessing operations

        return preprocessed_data

    def fit_arima_model(self, data):
        # Fit an ARIMA model to the data
        model = ARIMA(data, order=(p, d, q))
        model_fit = model.fit()
        return model_fit

    def generate_forecast(self, model):
        # Generate forecasted values using the fitted ARIMA model
        forecasted_values = model.forecast(steps=n_steps)
        return forecasted_values

    def placeOrder(self, symbol, right, strike, action, quantity):
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = "20231215"
        contract.strike = strike
        contract.right = right
        contract.multiplier = "100"

        order = Order()
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "LMT"
        order.lmtPrice = limit_price

        self.placeOrderWithContract(self.nextOrderId, contract, order)
        self.nextOrderId += 1

app = TradingApp()
app.connect("127.0.0.1", 4002, clientId=1)

# Define your options contract details
symbols = ["AAPL", "TSLA", "GOOGL"]
quantity = 1
threshold = 1.0  # Define your threshold value
limit_price = 0.0  # Define your limit price

# ARIMA model parameters
p = 1
d = 1
q = 1
n_steps = 1

# Request market data for the options contracts
for symbol in symbols:
    contract = Contract()
    contract.symbol = symbol
    contract.secType = "OPT"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.lastTradeDateOrContractMonth = "20231215"
    contract.strike = 150
    contract.right = "C"
    contract.multiplier = "100"

    app.reqMarketDataType(4)
    app.reqContractDetails(len(app.data), contract)
    app.reqMktData(len(app.data), contract, "", False, False, [])
    app.nextOrderId += 1

app.run()


ERROR:ibapi.wrapper:ERROR -1 502 Couldn't connect to TWS. Confirm that "Enable ActiveX and Socket EClients" 
is enabled and connection port is the same as "Socket Port" on the 
TWS "Edit->Global Configuration...->API->Settings" menu. Live Trading ports: 
TWS: 7496; IB Gateway: 4001. Simulated Trading ports for new installations 
of version 954.1 or newer:  TWS: 7497; IB Gateway: 4002
ERROR:ibapi.wrapper:ERROR -1 504 Not connected
ERROR:ibapi.wrapper:ERROR -1 504 Not connected
ERROR:ibapi.wrapper:ERROR 0 504 Not connected
ERROR:ibapi.wrapper:ERROR -1 504 Not connected
ERROR:ibapi.wrapper:ERROR -1 504 Not connected
ERROR:ibapi.wrapper:ERROR 0 504 Not connected
ERROR:ibapi.wrapper:ERROR -1 504 Not connected
ERROR:ibapi.wrapper:ERROR -1 504 Not connected
ERROR:ibapi.wrapper:ERROR 0 504 Not connected
